In [ ]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

In [ ]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis.shared import create_queries
%load_ext autoreload
%autoreload 2

In [ ]:
COMPANY_NAME = 'BBF'
COMPANY_IDENTIFIER = 'BBF'
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
    '10/31/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': True,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
            'Buds': {
                'grams': 10.0
            },
            'Infused (edible)': {
                'each': 4.0
            },
            'Infused (non-edible)': {
                'each': 3.0
            },
            'Vape Product': {
                'each': 3.0
            },
            'Concentrate (Bulk)': {
                'grams': 6.0
            },
            'Concentrate': {
                'grams': 7.0
            },
            'Raw Pre-Rolls': {
                'grams': 7.0,
                'pounds': 80.0
            },
            'Shake/Trim (by strain)': {
                'grams': 8.0
            }
        }
    }
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

In [ ]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER, include_quantity_zero=True)

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_sales_receipts_dataframe = pandas.read_sql_query(company_sales_receipts_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [ ]:
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
import importlib

importlib.reload(util)
importlib.reload(valuations_util)

In [ ]:
d = util.Download()
sql_helper = util.BigQuerySQLHelper(engine)
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
    sales_receipts_dataframe=company_sales_receipts_dataframe,
    inventory_packages_dataframe=company_inventory_packages_dataframe,
    sql_helper=sql_helper
)

In [ ]:
d.child_to_parent_package_id_override = {
#    '725714': '1687330'
}

In [ ]:
importlib.reload(util)
importlib.reload(valuations_util)

# Check for how many packages are excluded from analysis,
# seeing less then 2% is really good
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q, params=ANALYSIS_PARAMS)

In [ ]:
# For debugging individual package histories
# You have to run the above block to reload the package_id_to_history array

import importlib
importlib.reload(util)
importlib.reload(valuations_util)

# Missing transactions?
# 5189649

# Using grams or missing transactions?
#WARN: seeing an incoming package for #19083030 with no received_datetime
#Exception: FATAL error, could not find a transfer to insert a tx with date
#2021-10-22 11:58:45+00:00 into for package 18218608

# '2818807' - child

"""
WARN: incoming package #2358400 does not have a price
WARN: incoming package #1318689 does not have a price
WARN: incoming package #862836 does not have a price
WARN: incoming package #862838 does not have a price
WARN: incoming package #702114 does not have a price
WARN: incoming package #702301 does not have a price
"""

PACKAGE_IDS = [
  '862838'
]

util.analyze_specific_package_histories(
    d, package_id_to_actual_row, PACKAGE_IDS, params=ANALYSIS_PARAMS)

In [ ]:
util.run_orphan_analysis(d, id_to_history)

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(valuations_util)

#INVENTORY_DATES = [TODAY_DATE]

date_to_inventory_packages_dataframe = {}

id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
inventory_valuations = []

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=util.get_inventory_column_names()
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe
    inventory_valuations.append(valuations_util.get_total_valuation_for_date(
        computed_inventory_packages_dataframe=computed_inventory_packages_dataframe,
        company_incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe
    ))

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(valuations_util)

# Do the COGS summary too
topdown_cogs_rows = util.create_top_down_cogs_summary_for_all_dates(
    d, ANALYSIS_PARAMS
)

bottomsup_cogs_rows = util.create_cogs_summary_for_all_dates(
  id_to_history, ANALYSIS_PARAMS
)
util.write_cogs_xlsx(
    topdown_cogs_rows=topdown_cogs_rows, 
    bottoms_up_cogs_rows=bottomsup_cogs_rows,
    company_name=COMPANY_NAME
)

In [ ]:
for row in topdown_cogs_rows:
    print(row)

In [ ]:
for row in bottomsup_cogs_rows:
    print(row)

In [ ]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'unit_of_measure',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(valuations_util)

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 100,
        'accept_computed_when_sold_out': True
    }
)

# Two categories for: Num actual packages not computed: 229
# 1. Actual has a package that we've never seen
# 2. Actual has a package that we've sold out of, and that's why it's in the inventory

# Two categories for: Num computed packages not in actual: 237
# 1. We never saw the package in the actual inventory
# 2. Actual inventory is sold out, but we think it's not sold out in computed

In [ ]:
res

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(valuations_util)

valuations_util.plot_inventory_and_revenue(
    q=q,
    sales_receipts_dataframe=d.sales_receipts_dataframe,
    inventory_valuations=inventory_valuations
)

In [ ]:
inactive_packages_df = pandas.read_sql_query(
    util.are_packages_inactive_query(['766095']),
    engine
)
inactive_packages_df

In [ ]:
# Find transfer packages in data warehouse by package_id.

def create_transfer_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            company_deliveries.delivery_type,
            company_deliveries.updated_at,
            metrc_transfer_packages.package_id,
            metrc_transfer_packages.package_label,
            metrc_transfer_packages.shipped_quantity,
            metrc_transfer_packages.shipper_wholesale_price,
            metrc_transfers.shipper_facility_name,
            metrc_transfers.shipper_facility_license_number,
            metrc_deliveries.recipient_facility_name,
            metrc_deliveries.recipient_facility_license_number,
            metrc_transfer_packages.package_payload.*
        from
            metrc_transfer_packages
            inner join metrc_deliveries on metrc_transfer_packages.delivery_row_id = metrc_deliveries.id
            inner join metrc_transfers on metrc_deliveries.transfer_row_id = metrc_transfers.id
            inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
            inner join companies on company_deliveries.company_id = companies.id
        where
            True
            and metrc_transfer_packages.package_id = '{package_id}'
    """

transfer_packages_by_package_id_query = create_transfer_packages_by_package_id_query(
    2213735)
transfer_packages_by_package_id_dataframe = pandas.read_sql_query(transfer_packages_by_package_id_query, engine)
transfer_packages_by_package_id_dataframe

In [ ]:
transfer_packages_by_package_id_dataframe['receiverwholesaleprice']

In [ ]:
transfer_packages_by_package_id_dataframe.iloc[0].array

In [ ]:
transfer_packages_by_package_id_dataframe.columns

In [ ]:
# Find packages in data warehouse by package_id.

def create_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_packages.package_id,
            metrc_packages.package_label,
            metrc_packages.quantity,
            metrc_packages.package_payload.*
        from
            metrc_packages
            inner join companies on metrc_packages.company_id = companies.id
        where
            True
            and metrc_packages.package_id = '{package_id}'
    """

packages_by_package_id_query = create_packages_by_package_id_query(17481729)
packages_by_package_id_dataframe = pandas.read_sql_query(packages_by_package_id_query, engine)
packages_by_package_id_dataframe

In [ ]:
packages_by_package_id_dataframe.columns

In [ ]:
from bespoke.metrc.common import metrc_common_util

rest = metrc_common_util.get_rest_helper_for_debug(
    us_state='MA',
    license_number='MR281525'
)

In [ ]:
from dateutil import parser
from datetime import timedelta

start_date = parser.parse('2020/01/01').date()
end_date = parser.parse('2020/01/01').date()

cur_date = start_date

while cur_date <= end_date:
    cur_date_str = cur_date.strftime('%m/%d/%Y')
    resp = rest.get('/transfers/v1/rejected', time_range=[cur_date_str])
    transfers = json.loads(resp.content)
    if len(transfers) > 0:
        print('FOUND SOME')
        break
        
    cur_date = cur_date + timedelta(days=1)
    

In [ ]:
resp = rest.get('/packages/v1/967657')

In [ ]:
os.environ['JUPYTER_NOTEBOOK_METRC_API_KEY'] = ''

In [ ]:
os.environ['METRC_VENDOR_KEY_MA'] = ''